# 텍스트 정규화

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - nltk book 3.Processing raw text(https://www.nltk.org/book/ch03.html) 내용을 참고해서 자료를 만듦. 
    - nltk book의 3장 예제 인용.

### 텍스트 정규화
- 텍스트로부터 단어를 추출할 때, 유사한 단어들을 하나의 단어로 인식하기 위해 정규화가 필요합니다.
    - 단어를 소문자로 변환. 'The'와 'the'를 구분하지 않음.
    - 어간(stem)을 찾아 사전에 나오는 단어들로 변환(lemmatization - 표제어 추출).
- 또 다른 정규화는 비표준 단어(non-standard words)를 찾는 것입니다. 
    - 숫자, 날짜, 약어(acronym, ASCII, NASA 등) 등을 일반 단어와 구분해 찾는 것입니다.    

In [ ]:
import nltk, re, pprint

### Stemmers : Porter stemmers 와 Lancaster stemmers

In [ ]:
from nltk import word_tokenize

In [ ]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
 is no basis for a system of government.  Supreme executive power derives from
 a mandate from the masses, not from some farcical aquatic ceremony."""

tokens = word_tokenize(raw)

- Porter stemmer는 lying의 어간을 lie로 찾았지만, Lancaster stemmer는 lying으로 처리했습니다.
- 어간을 찾는 건 쉽지 않죠. 
    - strange -> strang, distributing -> distribut 등으로 제대로 찾지 못하는 경우가 더 많습니다.

In [ ]:
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()

print([porter.stem(t) for t in tokens])
print()
print([lancaster.stem(t) for t in tokens])

### IndexedText 클래스
- **concordance(self, word, width=40)** 
    - 단어 word를 기준으로 왼쪽에 놓인 단어 10개(*lcontext*), 오른쪽에 놓인 단어 10개(*rcontext*)를 찾음.
    - **ldisplay**: *lcontext*는 40개 padding을 기준으로 오른쪽 맞춤으로 출력.
    - **rdisplay**: *rcontext*는 40개 padding을 기준으로 왼쪽 맞춤으로 출력.
- **__init__(self, stemmer, text)**
    - **nltk.Index**는 튜플 타입의 원소를 갖는 리스트를 dictionary 객체로 변환. 
        - dictionary 객체의 key는 self.\_stem(word)가 되며, value는 인덱스 *i* 로 초기화됨.
        
        
- 눈치챘을지 모르겠지만, nltk 소개할 때 사용했던 concordance() 구현 방식과 같습니다.

In [ ]:
class IndexedText(object):

    def __init__(self, stemmer, text):
        self._text = text
        self._stemmer = stemmer
        self._index = nltk.Index((self._stem(word), i)
                                 for (i, word) in enumerate(text))

    def concordance(self, word, width=40):
        key = self._stem(word)
        wc = int(width/4)                # words of context
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            #print(lcontext) 
            #print(rcontext)         
            ldisplay = '{:>{width}}'.format(lcontext[-width:], width=width) #오른쪽 맞춤
            rdisplay = '{:{width}}'.format(rcontext[:width], width=width) #왼쪽 맞춤
            print(ldisplay, rdisplay)
            #print()

    def _stem(self, word):
        return self._stemmer.stem(word).lower() #어간을 구해 소문자 변환

단어 'from'은 텍스트에서 3번 나오기 때문에 인덱스 3개(24, 27, 32)가 추가되었습니다.

In [ ]:
tokens = word_tokenize(raw)
porter = nltk.PorterStemmer()

text = IndexedText(porter, tokens)
print(text._index)
print(text._index['from'])

- 단어 'from'에 대해 concordance 메소드를 실행시켜 보겠습니다.
    - concordance 메소드의 문장 중 print 문장 앞에 붙인 주석 기호를 모두 없애고 실행시켜 보세요.
    - lcontext, rcontext는 단어 10개를 갖고 있지만,
        - ldisplay, rdisplay는 40개 패딩, 즉 40개 문자만 출력합니다.

In [ ]:
text.concordance('from')

### 웹 말뭉치(webtext)
- 여러 곳에서 수집한 다양한 종류의 (규모가 크지 않은) 말뭉치.
    - Firefox discussion forum의 웹 텍스트, New York 시내에서 우연히 들은 대화 내용.
    - 캐러비안의 해적(Pirates of the Carribean)의 대본, 개인 홍보 자료, 와인 품평.

In [ ]:
porter = nltk.PorterStemmer()
grail = nltk.corpus.webtext.words('grail.txt')
print(grail[:50]) 

소문자로 변환한 후 Porter stemmer로 어간을 찾았기 때문에, text의 키는 Camelot가 아닌 camelot를 사용해야 합니다.

In [ ]:
text = IndexedText(porter, grail) 
print(text._index['wind'])
print(text._index['camelot']) 

Porter stemmer에서 lying의 어간은 lie로 변환하기 때문에 'lie'를 사용해야 합니다.

In [ ]:
text.concordance('lie')

##  WordNet lemmatizer
- 사전 원형 변환(Lemmatization). 표제어라고도 부릅니다.
    - 단어가 사전에 있으면 접미사(affix)를 없앱니다.
    - lemmatizer가 stemmer보다 사전 전체를 비교하므로 속도가 느립니다.
    - lying은 처리하지 못하지만 women은 woman으로 변환하네요.

앞에서 사용했던 원시 텍스트인데, 너무 멀리 떨어져 있어 아래 셀에 다시 복사해서 넣었습니다.

In [ ]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
 is no basis for a system of government.  Supreme executive power derives from
 a mandate from the masses, not from some farcical aquatic ceremony."""

tokens = word_tokenize(raw)

In [ ]:
wnl = nltk.WordNetLemmatizer()
wnl_list = [wnl.lemmatize(t) for t in tokens]
print(wnl_list)

In [ ]:
porter = nltk.PorterStemmer()
pt_list = [porter.stem(t) for t in tokens]
print(pt_list)

Wordnet lemmatizer와 Porter stemmer 결과를 비교해 보기 바랍니다.

In [ ]:
wnl_set = set(wnl_list)
pt_set = set(pt_list)
print(wnl_set.difference(pt_set))

## 정규표현을 사용한 토큰 찾기 

아래 원시 텍스트는 소설 이상한 나리의 앨리스(Alice's Adventures in Wonderland) 중 일부 문장입니다.

In [ ]:
raw = """'When I'M a Duchess,' she said to herself, (not in a very hopeful tone
though), 'I won't have any pepper in my kitchen AT ALL. Soup does very
well without--Maybe it's always pepper that makes people hot-tempered,'..."""

- 가장 간단한 토큰 찾는 방법은 split() 메소드를 사용하여 space 기호를 기준으로 나누면 됩니다.
    - 'When은 하나의 토큰이죠. 또, '...도 하나의 토큰으로 잘 인식되었군요.
    - 아... newline 기호('\n')가 2군데 몰래 포함되었군요.

In [ ]:
print(re.split(r' ', raw))

- 토큰 단위로 나누는 기준에 space기호 외에 '\t'과 '\n'기호를 모두 포함해서 해결할 수 있습니다.
    - 이 기호들을 whitespace라 부릅니다.

In [ ]:
print(re.split(r'[ \t\n]+', raw))

- 위에서 사용한 정규표현은 carriage-return(\r)이나 form-feed(\f)와 같은 whitespace 문자도 구분 기호로 잘 인식합니다.
- 그렇지만, 위 방식보다는 모든 whitespace 문자를 가리키는 메타 기호 '\s'를 사용하는 것이 좋습니다.
    - 훨씬 깔끔합니다. 전문가 포스가 물씬 풍기죠...

In [ ]:
print(re.split(r'\s+', raw))

### 단어 토큰을 찾기 위한 메타기호(\w)와 findall( )
- whitespace를 기준으로 토큰을 찾을 경우 '(not'나 'herself,' 처럼 구두점 기호나 괄호가 토큰에 포함됩니다.
    - 위 예에서도 쉽게 찾을 수 있습니다. 'When, though), ... 등이 해당하죠.
- 정규표현에서 제공하는 메타기호인 '\w'를 사용하면 이 문제를 쉽게 해결할 수 있습니다. 
    - '\w' = \[a-zA-Z0-9_\]. 즉, 숫자, 알파벳, underscore(\_)가 해당됩니다. 
    - '\W'는 '\w'의 정반대 기호들을 가리킵니다.

토큰을 찾기 위한 구분기호의 정규표현은 '\W+'를 사용하면 되겠죠...

In [ ]:
print(re.split(r'\W+', raw))

- 아니 이럴 수가... 다 끝난 줄 알았는데 맨 처음과 맨 끝에 빈 문자열(empty string)이 포함되어 있네요.
- 왜 이런 결과가 나왔을까요? 
    - split()의 기능 때문에 불필요한 빈 문자열이 포함되었군요...

In [ ]:
'xx'.split('x')

- 길게 돌아오긴 했지만, 결론은 다음과 같이 정리할 수 있습니다.
- split() 대신 findall()로 바꾸고, 정규표현의 메타기호도 '\w+'로 바꾸면 우리가 원하는 결과를 얻을 수는 있습니다.

In [ ]:
print(re.findall(r'\w+', raw))

### 메타기호: \w, \s, \W, \S 
정규표현을 보다 다양한 경우까지 고려하도록 확장시켜 보겠습니다.

In [ ]:
raw = """'When I'M a Duchess,' she said to herself, (not in a very hopeful tone
though), 'I won't have any pepper in my kitchen AT ALL. Soup does very
well without--Maybe it's always pepper that makes people hot-tempered,'..."""

- '\w+|\S\w*'
    - \w+는 문자로만 이루어진 토큰을 찾으려고 시도합니다.
    - 만약 찾지 못하면, 한 개의 non-whitespace 문자('\S') 다음에 문자가 나오는 경우(예:'s)를 찾습니다.
    - '\w*'이기 때문에 구두점 기호 한 개만 있어도 찾습니다. 
    - 2개 이상의 구두점 기호가 나오는 경우는 제외됩니다.

In [ ]:
print(re.findall(r'\w+|\S\w*', raw))

- '\w+|\S\w+'
    - '\S\w+'이기 때문에 구두점 기호 한 개와 단어가 나오는 경우만 토큰으로 인식합니다.

In [ ]:
print(re.findall(r'\w+|\S\w+', raw))

- 단어에 hyphens(-)이나 apostrophes(') 기호까지 함께 인식하도록 위 정규표현을 확장시켜 보겠습니다.
    - '\w+'를 '\w+([\-\']\w+)\*'로 바꿔보겠습니다.
    - 이렇게 하면 'hot-tempered'나 'it\'s'를 한 단어로 찾을 수 있습니다.
- 괄호를 단지 여러 패턴 조건을 묶기 위한 목적으로만 사용하고 이를 출력하지 않기 위해서는 '?:'를 앞에 붙여야 합니다.
    - 3-1.정규표현응용에서 이미 설명한 내용입니다.

In [ ]:
print(re.findall(r"\w+(?:[-']\w+)*|'|[-.(]+|\S\w*", raw))

### regexp_tokenize( )
- re.findall()과 기능은 거의 비슷합니다. 
- regexp_tokenize()가 조금 더 효율적이며, 특히 괄호 처리에 대한 고민을 덜어줍니다.
    - (?x)는 "verbose flag" 로써 주석이나 whitespace를 제거하라는 의미입니다.

In [ ]:
pattern = r'''(?x)      # flag : verbose - 주석 및 whitespace 제거
     (?:[A-Z]\.)+       # 요약(abbreviations) : U.S.A. 등
   | \w+(?:-\w+)*       # 단어 내부에 있는 hyphen : poster-print
   | \$?\d+(?:\.\d+)?%? # 화폐 단위 및 퍼센트 : $12.40, 82% 등
   | \.\.\.             # 생략 기호(ellipsis)
   | [][.,;"'?():-_`]   # 나머지 토큰들 : ], [ 등
'''
text = 'That U.S.A. poster-print costs $12.40...'

In [ ]:
nltk.regexp_tokenize(text, pattern)

findall() 메소드는 regexp_tokenize()와는 파라미터 순서가 다릅니다.

In [ ]:
re.findall(pattern, text)

##  Sentence segmentation : sent_tokenize()
- 텍스트를 단어 단위로 처리하려면, 먼저 텍스트를 문장(sentence) 단위로 나누어야 합니다. 
- 먼저 문장 단위로 미리 나누어져 있는 말뭉치부터 살펴보겠습니다. 

Brown 말뭉치에서 문장 당 평균 단어 수를 구해보겠습니다.

In [ ]:
avg_words = len(nltk.corpus.brown.words()) / len(nltk.corpus.brown.sents())
print(round(avg_words, 4))

In [ ]:
print(nltk.corpus.brown.sents()[0]) #첫번째 문장 출력

Gutenberg 말뭉치 중 G. K. Chesterton 의 '목요일이라 불리운 사나이'(The Man Who Was Thursday)를 사용하겠습니다.

In [ ]:
text = nltk.corpus.gutenberg.raw('chesterton-thursday.txt') 
text[:50]

sent_tokenize()를 사용하여 텍스트를 문장 단위로 나눌 수 있습니다.

In [ ]:
sents = nltk.sent_tokenize(text)
pprint.pprint(sents[79:89])

- 위 문장은 사실 놀랍게도 하나의 문장입니다(?). 
    - 여러 대화가 인용문 형태로 포함되어 있기 때문에 별개 문장으로 구분할 수 있습니다. 
- 문장을 어떻게 구분할 것인가는 응용 목적에 따라 다릅니다. 
    - 문장 단위로 나누는 것도 쉬운 일은 아닙니다.
    - 마침표('.')가 'U.S.A.'처럼 요약을 위해 사용되기 때문에, 마침표만으로 무조건 문장을 구분할 수는 없습니다.

## Word segmentation
- 단어와 단어 사이에 구분 기호 없이 쭉~ 연결되어 있는 경우 어떻게 단어들을 구분해 낼 수 있을까요? 
    - 'doyouseethekitty', 'seethedoggy' 처럼 주로 대화 중에 상대방이 쭉 이어서 말하는 경우가 대표적 예입니다.
    - 화자(speaker)가 발음 중간에 잠깐 쉬어가는 틈(word-break)을 부울 값으로 표시하면 구분이 가능하겠죠.
    - 아래 예에서 각 알파벳마다 0은 연음으로, 1은 쉬어가는 틈을 나타냅니다.

In [ ]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"

In [ ]:
def segment(text, segs):
    words = []
    last = 0
    for i in range(len(segs)):
        if segs[i] == '1':
            words.append(text[last:i+1])
            last = i+1
    words.append(text[last:])
    return words

연음과 쉬어가는 틈을 표기한 기호가 2가지 있죠. 먼저 seg1을 적용해서 단어를 쪼개보겠습니다.

In [ ]:
segment(text, seg1)

이번에는 seg2를 적용해서 단어를 쪼개보겠습니다.

In [ ]:
print(segment(text, seg2))

- word segmentation 작업은 탐색 문제로 바꿀 수 있습니다. 
    - 즉, 텍스트를 단어 단위로 정확히 쪼개기 위해 필요한 최소 비트열을 구하는 최적화 탐색 문제입니다.
    - 가정 : 학습자(또는 듣는 사람)는 단어를 획득해 lexicon에 저장해 놓았다고 가정하겠습니다.
        - (doyouseethekitty -> doyou, see, thekitt, y), (seethedoggy -> see, thedogg, y), 
        - (doyoulikethekitty -> doyou, like, thekitt, y), (likethedoggy -> like, thedogg, y)
        - lexcon은 6개 단어를 저장 : 1)doyou, 2)see, 3)like, 4)thekitt, 5)thedogg, 6)y
    - lexicon이 있으면, lexicon에 속한 원소들을 순서대로 나열해 원본 텍스트를 재구성하는 것이 가능합니다.
- 즉 lexicon으로부터 원본 텍스트 재구성에 필요한 정보를 토대로 목적 함수(objective function)을 정의할 수 있으며, 
     - 이를 최적화하면 됩니다.

- 적용 과정
- 가정: 문장을 단어로 분할한 결과.
     - 'doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy'
- 가정: 6개 lexicon(단어사전)도 저장되어 있음.
    - 1)doyou, 2)see, 3)like, 4)thekitt, 5)thedogg, 6)y
- 유도(derivation): 
    - doyou, see, thekitt, y -> 1 2 4 6 
    - see, thedogg, y -> 2, 5, 6
    - doyou, like, thekitt, y -> 1, 3, 4, 6
    - like, thedogg, y -> 3, 5, 6
4. 원본 재구성에 필요한 정보와 목적 함수
    - lexicon : 6 + 4 + 5 + 8 + 8 + 2 = 33
        - 단어 구분기호를 포함하면 단어 길이+1. 5(doyou)+1=6, 3(see)+1=4, 4(like)+1=5, ...
    - derivation : 4 + 3 + 4 + 3 = 14
        - doyou, see, thekitt, y -> 1 2 4 6 -> 4(lexicon 수) 
        - see, thedogg, y -> 2, 5, 6 -> 3, ...    
    - 목적함수 : total = 33(lexicon) + 14(derivation) = 47 
        -  이 값을 최적화하기 위한 인코딩 방식을 찾음.

In [ ]:
def evaluate(text, segs):
    words = segment(text, segs)
    text_size = len(words)
    lexicon_size = sum(len(word) + 1 for word in set(words))
    return text_size + lexicon_size

In [ ]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"
seg3 = "0000100100000011001000000110000100010000001100010000001"

3가지 인코딩 방식 중에 seg3가 가장 작은 값(47)을 갖습니다. 우리는 seg3와 같은 인코딩 패턴을 찾아야 합니다.

In [ ]:
segment(text, seg3)
print(evaluate(text, seg3))
print(evaluate(text, seg2))
print(evaluate(text, seg1))

### Non-Deterministic Search Using Simulated Annealing(SA) 
- SA는 온도를 서서히 낮춰가며(cooling rate)가며 최적해를 찾는 방식
    - 초기에 1과 0을 랜덤하게 뒤썩은 상태에서 단어를 쪼갭니다. 
    - 온도를 낮춰가며 최적 구분 경계를 찾습니다. 
    - 온도가 높을수록 값이 바뀌는 대상이 많지만, 온도가 낮아질수록 값이 바뀌는 대상이 줄어듭니다.    

In [ ]:
from random import randint

#segs[pos]의 값을 반대로 바꿈: 0->1, 1->0.
def flip(segs, pos):
    return segs[:pos] + str(1-int(segs[pos])) + segs[pos+1:]

#모두 n개의 원소 값을 바꿈.
def flip_n(segs, n):
    for i in range(n):
        segs = flip(segs, randint(0, len(segs)-1))
    return segs

In [ ]:
seg1 = "0000000000000001000000000010000000000000000100000000000"
print(len(seg1), randint(0, len(seg1)-1))
flip(seg1, randint(0, len(seg1)-1))
flip_n(seg1, (len(seg1)))

In [ ]:
def anneal(text, segs, iterations, cooling_rate):
    temperature = float(len(segs))
    while temperature > 0.5:
        best_segs, best = segs, evaluate(text, segs)
        for i in range(iterations):
            guess = flip_n(segs, round(temperature))
            score = evaluate(text, guess)
            if score < best:
                best, best_segs = score, guess
        score, segs = best, best_segs
        temperature = temperature / cooling_rate
        print('%1.2f' % temperature, end=' ')
        print(evaluate(text, segs), segment(text, segs))
    print()
    return segs

온도가 낮아질수록 비용함수가 줄어들며 최적 인코딩을 찾아갑니다.

In [ ]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
anneal(text, seg1, 5000, 1.2)